# 🔥 AI & Maskininlärning: Brandbildsdetektion
### En praktisk demonstration av Supervised Learning och LLM för bildanalys

## 📌 Ladda in dataset & MobileNetV2
**Mål:**
- Ladda MobileNetV2 och förbereda träningsdata
- Visa datasetstruktur och vilka bilder som används
- Kontrollera om GPU används

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import os
from PIL import Image
import psutil

import psutil

# Kolla batteristatus
battery = psutil.sensors_battery()
if battery and battery.power_plugged is False:
    print("⚠️ Warning: Your laptop is running on battery. Performance may be reduced, "
          "and CUDA might fail due to power-saving features.")

# Check CUDA availability
if torch.cuda.is_available():
    print(f"✅ CUDA is available: Running on {torch.cuda.get_device_name(0)}")
else:
    print("❌ CUDA is not available: Running on CPU. Ensure your GPU drivers are installed correctly.")


# Kolla om CUDA finns tillgängligt
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(f'✅ Using device: {device}')

# Ladda MobileNetV2
weights = models.MobileNet_V2_Weights.DEFAULT
model = models.mobilenet_v2(weights=weights)
print("✅ Modell laddad!")  # Detta dämpar den automatiska utskriften


# Anpassa modellen för binär klassificering
model.classifier[1] = nn.Linear(model.last_channel, 2)
model.to(device)

## 🟠 Träna modellen (72+72 bilder)
**Mål:**
- Vårt indata
  - 655 brandbilder
  - 144 icke brandbilder
- Träna modellen i 10 epoker
  - Epoch 1: Modellen gissar nästan slumpmässigt.
  - Epoch 5: Modellen börjar förstå skillnaden mellan eld och solnedgång.
  - Epoch 10: Noggrannheten ökar, men modellen kan fortfarande göra vissa misstag.
  - Epoch 20+: Risken finns att modellen blir övertränad och bara "memorerar" träningsdata istället för att förstå mönster.
- Visa hur modellen lär sig genom att logga loss & accuracy
- Spara den tränade modellen

In [ ]:
# Definiera datatransformationer
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Anpassad datasetklass
class FireDataset(torch.utils.data.Dataset):
    def __init__(self, root, transform=None):
        self.fire_path = os.path.join(root, 'fire')
        self.non_fire_path = os.path.join(root, 'non_fire')
        self.transform = transform
        self.images = []
        for img_file in os.listdir(self.fire_path):
            self.images.append((os.path.join(self.fire_path, img_file), 1))
        for img_file in os.listdir(self.non_fire_path):
            self.images.append((os.path.join(self.non_fire_path, img_file), 0))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path, label = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

## Vi bestämmer vilket data vi ska använda

In [ ]:
# Ladda dataset
train_dataset = FireDataset('dataset/train', transform=transform)
val_dataset = FireDataset('dataset/val', transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=False)

# Träningsloop
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    
    train_acc = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}, Train Acc: {train_acc:.2f}%')

# Spara modellen
torch.save(model.state_dict(), 'fire_classifier.pth')
print('✅ Modell sparad!')

## 🔴 Testa modellen & Confusion Matrix
**Mål:**
- Ladda den tränade modellen
- Testa modellen på hela datasetet
- Visa Confusion Matrix

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Ladda modellen och sätt i eval-mode
model.load_state_dict(torch.load('fire_classifier.pth', map_location=device))
model.eval()

# Testa modellen
true_labels = []
pred_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        true_labels.extend(labels.cpu().numpy())
        pred_labels.extend(predicted.cpu().numpy())

# Skapa Confusion Matrix
cm = confusion_matrix(true_labels, pred_labels)
class_names = ['non_fire', 'fire']
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()